# Modèle quasi-géostrophique à tourbillon potentiel uniforme

- Modèle Scilab : Matthieu Plu (GMAP/RECYF, Nov 2005)
- Portage Python 3 : Clément Blot & Guillem Coquelet (IENM3, Septembre 2014 - Janvier 2015)
- Mises à jour : Frédéric Ferry (ENM/C3M, février 2021)

<div class="alert alert-warning">
<b>Instructions : </b>
<p><b>1) </b>Exécuter les cellules qui suivent de façon séquentielle</p>
<p><b>2) </b>Répondre aux questions (cadres de couleur jaune) dans les parties dédiées (cadres de couleur verte)</p>
<p><b>3) </b>Sauvegarder le notebook final au format pdf</p>
</div>

In [ ]:
import sys
import warnings
warnings.filterwarnings("ignore")

import numpy as np
from numpy import amax, amin, array, linspace, shape, transpose
from numpy.ma.core import shape, zeros

import matplotlib.pyplot as plt
import matplotlib.path as mpath

import IPython.display as IPdisplay, matplotlib.font_manager as fm
from PIL import Image
import glob

# Modèle QG
from config import DT, H, LX, LY, N, M, MUTIL, NUTIL
from model.etat_initial import EtatInitial
from model.modele import Modele

from model.diagnostic import diag_h_theta, diag_h_vorti, diag_h_geop, \
    diag_h_vv, diag_h_wind, diag_h_deform, diag_h_gradtheta, diag_h_q, \
    diag_m_theta, diag_m_wind, diag_z_theta, diag_z_vorti, diag_z_vv

In [ ]:
#################### Configuration du modèle  #########################
#
# Dimension x (m) : LX
# Dimension y (m) : LY
# Dimension z (m) : H
# Pas de temps (s) : DT
# Nb de points x (lat) : MUTIL
# Nb de points y (lon) : NUTIL
#
#################### Etat initial du modèle ###############
#
# thhi, thbi = EtatInitial().gradeady_gradwernli()
# steps = [0]
# thh, thb = Modele().run(thhi, thbi, steps)
#
#################### Lancement du modèle avec superposition des anomalies ###############
#
# thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1400)
# steps = [0, 48, 96, 144, 192, 240, 288, 336, 384]
# thh, thb = Modele().run(thhi, thbi, steps)
#
################## Niveaux du modèle ################
#
# Niveau bas du modèle : z=-H/2
# Niveau haut du modèle : z=H/2
# Niveau VV près du sol : z=-H/4
# Niveau VV près de la tropopause : z=H/4
#
################## Coupes horizontales utilisées ######################
#
# diag_h_geop(thh, thb, z) : géopotentiel
# diag_h_theta(thh, thb, z) : température potentielle
# diag_h_theta(thh, thb, z) : ug, vg - vent géostrophique
# diag_h_vorti(thh, thb, z) : tourbillon géostrophique
# diag_h_vv(thh, thb, z) : vitesse verticale
# diag_h_deform(thh, thb, z) : d1, d2 - déformation d'étirement et de cisaillement
# diag_gradtheta(thh, thb, z) : module du gradient horizontal de theta
# diag_h_q(thh, thb, z) : Q1, Q2, div(Q) - forçage géostrophique et divergence
#
################## Coupes méridiennes utilisées #######################
#
# diag_m_theta(thh, thb, y, n_z) : température potentielle
# diag_m_wind(thh, thb, y, n_z) : ug, vg - vent géostrophique
#
################## Coupes zonales utilisées ###########################
#
# diag_z_theta(thh, thb, x, n_z) : température potentielle
# diag_z_vorti(thh, thb, x, n_z) : tourbillon géostrophique
# diag_z_vv(thh, thb, x, n_z) : vitesse verticale
#
#######################################################################

In [ ]:
dir_figs='./out/'

In [ ]:
def make_animation():
    nbimages=len(steps)
    # create a tuple of display durations, one for each frame
    first_last = 1000 #show the first and last frames for 100 ms
    standard_duration = 1000 #show all other frames for 5 ms
    durations = tuple([first_last] + [standard_duration] * (nbimages - 2) + [first_last])
    # load all the static images into a list
    images = [Image.open(image) for image in sorted(glob.glob('{}/*.png'.format(dir_anim)))]
    # save as an animated gif
    gif = images[0]
    gif.info['duration'] = durations #ms per frame
    gif.info['loop'] = 0 #how many times to loop (0=infinite)
    gif.save(fp=gif_filepath, format='gif', save_all=True, append_images=images[1:])
    # verify that the number of frames in the gif equals the number of image files and durations
    Image.open(gif_filepath).n_frames == len(images) == len(durations)
    # clean png
    #os.chdir(dir_anim)
    #for f in glob.glob("*.png"):
    #    os.remove(f)
    #os.chdir("../../")
    return Image

In [ ]:
print("Dimension x (m) : ",LX)
print("Dimension y (m) : ",LY)
print("Dimension z (m) : ",H)
print("Pas de temps (s) : ",DT)
print("Nb de points x (lat) : ",MUTIL)
print("Nb de points y (lon) : ",NUTIL)

abs = array(range(0,int(LX * 1e-3 + MUTIL),int(LX * 1e-3 / (MUTIL - 1) + 1)))
ord = array(range(0,int(LY * 1e-3 + NUTIL),int(LY * 1e-3 / (NUTIL - 1) + 1)))
print(abs)
print(ord)

In [ ]:
def plot_background():
    plt.gca().invert_yaxis()
    plt.grid(True)  # Ajout de la grille
    plt.xlabel("Axe Ouest - Est ($km$)", fontsize=10)
    plt.ylabel("Axe Nord - Sud ($km$)", fontsize=10)

In [ ]:
levels_u2=linspace(0, 10, 11) # vent zonal Eady
levels_u=linspace(-40, 40, 17) # vent zonal modèle
levels_th_m=linspace(280, 340, 13) # theta modèle coupe verticale
levels_th_anom=linspace(-7, 7, 15) # anomalies de température potentielle
levels_geo_h=linspace(-700, 700, 36) # geopotentiel coupe horizontale
levels_th_h=linspace(280, 340, 21) # theta coupe horizontale modèle
levels_vort_h_pos=linspace(1e-5, 1.01e-4, 11) # tourbillon geostrophique+ coupe horizontale
levels_vort_h_neg=linspace(-1.01e-4, -1e-6, 11) # tourbillon geostrophique- coupe horizontale
levels_vort_z_pos=linspace(0, 1e-4, 21) # tourbillon geostrophique+ coupe zonale
levels_vort_z_neg=linspace(-1e-4, -5e-6, 20) # tourbillon geostrophique- coupe zonale
levels_vv_h_neg=linspace(-0.022, -0.002, 11) # VV-
levels_vv_h_pos=linspace(0.002, 0.022, 11) # VV+
levels_modth_h=linspace(8e-8, 5e-5, 20) # Module du gradient horizontal de température potentielle
levels_divQ_h=linspace(-30e-18, 30e-18, 31) # divergence de Q

# Question 1 : Etat de base Eady

In [ ]:
thhi, thbi = EtatInitial().gradeady()
steps = [0]
thh, thb = Modele().run(thhi, thbi, steps)

for i in range(shape(thh)[0]):
    th1 = []
    th2 = []
    
    th1.append(diag_h_theta(thh[i], thb[i], z=-H / 2))
    th2.append(diag_h_theta(thh[i], thb[i], z=H / 2))    
    th12 = np.reshape(th1, (MUTIL, NUTIL))
    th22 = np.reshape(th2, (MUTIL, NUTIL))
    
    y = 8e6
    n_z = 11
    
    theta = []
    ug = []
    theta.append(transpose(diag_m_theta(thh[i], thb[i], y, n_z)))
    ug.append(transpose(diag_m_wind(thh[i], thb[i], y, n_z)[1]))
    
    theta2 = np.reshape(theta, (n_z, MUTIL))
    ug2 = np.reshape(ug, (n_z, MUTIL))
    print(theta2.shape)
    print(ug2.shape)

In [ ]:
fig = plt.figure(figsize=(20,15))
fig.suptitle('Etat de base du modèle : coupe horizontales sol/tropopause', fontsize=16)

plt.subplot(221)
plt.title("Température potentielle au sol")
plot_background()
c = plt.contour(ord,abs,th12,levels=levels_th_h,linewidth=1.5)
plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")

plt.subplot(222)
plt.title("Température potentielle à la tropopause")
plot_background()
c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")

plt.subplot(223)
plt.title('Coupe méridienne température potentielle et vent zonal')
z = range(0, int(H + 1), int(H / (n_z - 1)))
plt.xlabel("Axe Nord - Sud ($km$)", fontsize=10)
plt.xticks(np.arange(MUTIL)[::12],abs[::12])
plt.ylabel("Altitude ($km$)", fontsize=10)
c = plt.contour(theta2,z,levels=levels_th_m,linewidths=2)
plt.clabel(c,fontsize=8,fmt="%1.0f")
cf = plt.contourf(ug2,z,levels=levels_u2,cmap="Reds")
plt.clabel(cf,inline=1,fontsize=8,fmt="%1.0f",colors="gray")
colord = plt.colorbar(cf, orientation="vertical")
colord.set_label("Vent géostrophique zonal $(m.s^{-1})$")

figname=dir_figs+'Q1_Eady'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Décrire l'état de base "Eady".</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Question 2 : Etat de base du modèle

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli()
steps = [0]
thh, thb = Modele().run(thhi, thbi, steps)

geo1 = []
geo2 = []
th1 = []
th2 = []
vort1 = []
vort2 = []
ug1,vg1 = [], []
ug2,vg2 = [], []

geo1.append(diag_h_geop(thh, thb, z=-H / 2))
geo2.append(diag_h_geop(thh, thb, z=H / 2))
th1.append(diag_h_theta(thh, thb, z=-H / 2))
th2.append(diag_h_theta(thh, thb, z=H / 2))
vort1.append(diag_h_vorti(thh, thb, z=-H / 2))
vort2.append(diag_h_vorti(thh, thb, z=H / 2))
ug_tmp, vg_tmp = diag_h_wind(thh, thb, z=-H / 2)
ug1.append(ug_tmp)
vg1.append(vg_tmp)
ug_tmp, vg_tmp = diag_h_wind(thh, thb, z=H / 2)
ug2.append(ug_tmp)
vg2.append(vg_tmp)
    
geo12 = np.reshape(geo1, (MUTIL, NUTIL))
geo22 = np.reshape(geo2, (MUTIL, NUTIL))
th12 = np.reshape(th1, (MUTIL, NUTIL))
th22 = np.reshape(th2, (MUTIL, NUTIL))
vort12 = np.reshape(vort1, (MUTIL, NUTIL))
vort22 = np.reshape(vort2, (MUTIL, NUTIL))
ug12 = np.reshape(ug1, (MUTIL, NUTIL))
vg12 = np.reshape(vg1, (MUTIL, NUTIL))
ug22 = np.reshape(ug2, (MUTIL, NUTIL))
vg22 = np.reshape(vg2, (MUTIL, NUTIL))

In [ ]:
fig = plt.figure(figsize=(17,15))
fig.suptitle('Etat de base du modèle : coupe horizontales sol/tropopause', fontsize=16)

plt.subplot(321)
plt.title("Géopotentiel et vent au sol")
plot_background()
c = plt.contour(ord,abs,geo12,levels_geo_h)
plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
plt.quiver(ord[::5],abs[::5],vg12[::5, ::5],-ug12[::5, ::5],scale=500,headwidth=8,width=0.0013)

plt.subplot(322)
plt.title("Géopotentiel et vent à la tropopause")
plot_background()
c = plt.contour(ord,abs,geo22,levels_geo_h)
plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
plt.quiver(ord[::5],abs[::5],vg22[::5, ::5],-ug22[::5, ::5],scale=500,headwidth=8,width=0.0013)

plt.subplot(323)
plt.title("Température potentielle et vent au sol")
plot_background()
c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
plt.quiver(ord[::5],abs[::5],vg12[::5, ::5],-ug12[::5, ::5],scale=500,headwidth=8,width=0.0013)

plt.subplot(324)
plt.title("Température potentielle et vent à la tropopause")
plot_background()
c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
plt.quiver(ord[::5],abs[::5],vg22[::5, ::5],-ug22[::5, ::5],scale=500,headwidth=8,width=0.0013)

plt.subplot(325)
plt.title("Tourbillon géostrophique et vitesse verticale au sol")
plot_background()
c1 = plt.contour(ord,abs,vort12,levels_vort_h_pos,colors="blue")
plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
c2 = plt.contour(ord,abs,vort12,levels_vort_h_neg,colors="red")
plt.clabel(c2,c2.levels[::2],inline=1,fontsize=6,fmt="%1.6f")

plt.subplot(326)
plt.title("Tourbillon géostrophique et vitesse verticale à la tropopause")
plot_background()
c1 = plt.contour(ord,abs,vort22,levels_vort_h_pos,colors="blue")
plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
c2 = plt.contour(ord,abs,vort22,levels_vort_h_neg,colors="red")
plt.clabel(c2,c2.levels[::2],inline=1,fontsize=6,fmt="%1.6f")

figname=dir_figs+'Q2_base'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

In [ ]:
y = 8e6
n_z = 11

for i in range(shape(thh)[0]):
    theta = []
    ug = []
    theta.append(transpose(diag_m_theta(thh[i], thb[i], y, n_z)))
    ug.append(transpose(diag_m_wind(thh[i], thb[i], y, n_z)[1]))
    
    theta2 = np.reshape(theta, (n_z, MUTIL))
    ug2 = np.reshape(ug, (n_z, MUTIL))
    print(theta2.shape)
    print(ug2.shape)

In [ ]:
fig = plt.figure(figsize=(12,8))
fig.suptitle('Etat de base du modèle', fontsize=16)

plt.subplot(111)
plt.title('Coupe méridienne température potentielle et vent zonal')
z = range(0, int(H + 1), int(H / (n_z - 1)))
plt.xlabel("Axe Nord - Sud ($km$)", fontsize=10)
plt.xticks(np.arange(MUTIL)[::12],abs[::12])
plt.ylabel("Altitude ($km$)", fontsize=10)
c = plt.contour(theta2,z,levels=levels_th_m,linewidths=2)
plt.clabel(c,fontsize=8,fmt="%1.0f")
cf = plt.contourf(ug2,z,levels=levels_u,cmap="seismic")
plt.clabel(cf,inline=1,fontsize=8,fmt="%1.0f",colors="gray")
colord = plt.colorbar(cf, orientation="vertical")
colord.set_label("Vent géostrophique zonal $(m.s^{-1})$")

figname=dir_figs+'Q2_zone_barocline'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Décrire l'état de base du modèle.</p>
<p><b>2) </b>Quantifier la baroclinie en milieu de domaine en termes de gradient méridien de température et de cisaillement de vent zonal correspondant. Comment qualifier cette baroclinie par rapport à la réalité ?.</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Question 3 : Etat initial

In [ ]:
steps = [0]
tilt = 1400

thhi_a, thbi_a = EtatInitial().anotripo(tilt)
thh_a, thb_a = Modele().run(thhi_a, thbi_a, steps)
th1_a = []
th2_a = []
th1_a.append(diag_h_theta(thh_a, thb_a, z=-H / 2))
th2_a.append(diag_h_theta(thh_a, thb_a, z=H / 2))
th12_a = np.reshape(th1_a, (MUTIL, NUTIL))
th22_a = np.reshape(th2_a, (MUTIL, NUTIL))

thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt)
thh, thb = Modele().run(thhi, thbi, steps)
th1 = []
th2 = []
th1.append(diag_h_theta(thh, thb, z=-H / 2))
th2.append(diag_h_theta(thh, thb, z=H / 2))
th12 = np.reshape(th1, (MUTIL, NUTIL))
th22 = np.reshape(th2, (MUTIL, NUTIL))

In [ ]:
fig = plt.figure(figsize=(20,10))
fig.suptitle('Etat initial du modèle', fontsize=16)

plt.subplot(221)
plt.title("Anomalie de température potentielle au sol")
plot_background()
c = plt.contour(ord,abs,th12_a,levels_th_anom,linewidth=1.5)
plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")

plt.subplot(222)
plt.title("Etat initial en température potentielle au sol")
plot_background()
c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")

plt.subplot(223)
plt.title("Anomalie de température potentielle à la tropopause")
plot_background()
c = plt.contour(ord,abs,th22_a,levels_th_anom,linewidth=1.5)
plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")

plt.subplot(224)
plt.title("Etat initial en température potentielle à la tropopause")
plot_background()
c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")

figname=dir_figs+'Q3_anomT'
plt.savefig(figname+'.png',bbox_inches='tight')
plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Caractériser les perturbations de température superposées à l'état de base (forme, signe, intensité, localisation).</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Question 4 : Cyclogenèse avec 2 anomalies - instant initial

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1400)
steps = [0]
thh, thb = Modele().run(thhi, thbi, steps)

for i in range(shape(thh)[0]):
    
    geo1 = []
    geo2 = []
    th1 = []
    th2 = []
    vort1 = []
    vort2 = []
    vv1 = []
    vv2 = []
    ug1,vg1 = [], []
    ug2,vg2 = [], []

    geo1.append(diag_h_geop(thh[i], thb[i], z=-H / 2))
    geo2.append(diag_h_geop(thh[i], thb[i], z=H / 2))
    th1.append(diag_h_theta(thh[i], thb[i], z=-H / 2))
    th2.append(diag_h_theta(thh[i], thb[i], z=H / 2))
    vort1.append(diag_h_vorti(thh[i], thb[i], z=-H / 2))
    vort2.append(diag_h_vorti(thh[i], thb[i], z=H / 2))
    vv1.append(diag_h_vv(thh[i], thb[i], z=-H / 4))
    vv2.append(diag_h_vv(thh[i], thb[i], z=H / 4))
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=-H / 2)
    ug1.append(ug_tmp)
    vg1.append(vg_tmp)
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=H / 2)
    ug2.append(ug_tmp)
    vg2.append(vg_tmp)
    
    geo12 = np.reshape(geo1, (MUTIL, NUTIL))
    geo22 = np.reshape(geo2, (MUTIL, NUTIL))
    th12 = np.reshape(th1, (MUTIL, NUTIL))
    th22 = np.reshape(th2, (MUTIL, NUTIL))
    vort12 = np.reshape(vort1, (MUTIL, NUTIL))
    vort22 = np.reshape(vort2, (MUTIL, NUTIL))
    vv12 = np.reshape(vv1, (MUTIL, NUTIL))
    vv22 = np.reshape(vv2, (MUTIL, NUTIL))
    ug12 = np.reshape(ug1, (MUTIL, NUTIL))
    vg12 = np.reshape(vg1, (MUTIL, NUTIL))
    ug22 = np.reshape(ug2, (MUTIL, NUTIL))
    vg22 = np.reshape(vg2, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(20,10))
    fig.suptitle('Etat initial du modèle : coupes horizontales sol/tropopause', fontsize=16)
    
    plt.subplot(221)
    plt.title("Géopotentiel température potentielle et vent au sol")
    plot_background()
    c = plt.contour(ord,abs,geo12,levels_geo_h,colors='k')
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    plt.quiver(ord[::5],abs[::5],vg12[::5, ::5],-ug12[::5, ::5],scale=500,headwidth=8,width=0.0013)
    
    plt.subplot(222)
    plt.title("Géopotentiel température potentielle et vent à la tropopause")
    plot_background()
    c = plt.contour(ord,abs,geo22,levels_geo_h,colors='k')
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    plt.quiver(ord[::5],abs[::5],vg22[::5, ::5],-ug22[::5, ::5],scale=500,headwidth=8,width=0.0013)
    
    plt.subplot(223)
    plt.title("Tourbillon géostrophique température potentielle et vitesse verticale au sol")
    plot_background()
    c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c1 = plt.contour(ord,abs,vort12,levels_vort_h_pos,colors="blue")
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv12,levels_vv_h_neg,colors="black",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv12,levels_vv_h_pos,colors="black",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    plt.subplot(224)
    plt.title("Tourbillon géostrophique et température potentielle et vitesse verticale à la tropopause")
    plot_background()
    c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c1 = plt.contour(ord,abs,vort22,levels_vort_h_pos,colors="blue")
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv22,levels_vv_h_neg,colors="black",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv22,levels_vv_h_pos,colors="black",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    figname=dir_figs+'Q4_init_h'
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.show()
    
    # Ajout d'une coupe zonale
    x = 4e6
    n_z = 11
    dz = H / (n_z - 1)
    z1 = range(0, int(H + dz), int(dz))
    
    theta = []
    vorti = []

    theta.append(transpose(diag_z_theta(thh[i], thb[i], x, n_z)))
    vorti.append(transpose(diag_z_vorti(thh[i], thb[i], x, n_z)))

    theta2 = np.reshape(theta, (n_z, NUTIL))
    vorti2 = np.reshape(vorti, (n_z, NUTIL))  

    n_z = 13
    dz = H / (n_z - 1)
    z2 = range(0, int(H + dz), int(dz))
    
    vvv = []
    vvv.append(transpose(diag_z_vv(thh[i], thb[i], x, n_z)))
    
    vvv2 = np.reshape(vvv, (n_z, NUTIL))
    
    fig = plt.figure(figsize=(10,8))
    fig.suptitle('Etat initial du modèle : coupe zonale', fontsize=16)
    plt.subplot(111)
    plt.title("Température potentielle tourbillon géostrophique et vitesse verticale")
    plt.ylim([0, 10])
    plt.xlabel("Axe Ouest - Est ($km$)", fontsize=10)
    plt.xticks(np.arange(NUTIL)[::12],ord[::12])
    plt.ylabel("Altitude ($km$)", fontsize=10)
    c = plt.contour(theta2,levels=levels_th_m,linewidths=2)
    plt.clabel(c, fontsize=8, fmt="%1.0f")
    #c1 = plt.contour(vorti2,levels=levels_vort_z_neg,colors='red',linewidths=2)
    #plt.clabel(c1, c1.levels[::2], fontsize=8, fmt="%.0e")
    c2 = plt.contour(vorti2,levels=levels_vort_z_pos,colors='blue',linewidths=2)
    plt.clabel(c2, c2.levels[::2], fontsize=8, fmt="%.0e")
    #plt.quiver(z2[::4],ord[::4],vvv2[::4, ::4] *1e-50,vvv2[::4, ::4] *1000,
    #           scale=50,width=0.005,headwidth=3,scale_units="width")
    c = plt.contour(vvv2, colors='k')
    plt.clabel(c, c.levels[::2], fontsize=7, fmt="%.0e")
    
    figname=dir_figs+'Q4_init_z'
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.show()

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Vérifier que les perturbations de tourbillon, de température et de stabilité statique répondent à ce que l'on soit avoir dans une atmosphère à tourbillon potentiel uniforme.</p>
<p><b>2) </b>Comment expliquer la présence de noyaux de vitesses verticales à l'instant initial ?</p> 
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Question 4 : Cyclogenèse avec 2 anomalies - simulation sur 4 jours

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1400)
steps = [0, 48, 96, 144, 192, 240, 288, 336, 384]

thh, thb = Modele().run(thhi, thbi, steps)

mini = []
mini_ind = []

for i in range(shape(thh)[0]):
    
    geo1 = []
    geo2 = []
    th1 = []
    th2 = []
    vort1 = []
    vort2 = []
    vv1 = []
    vv2 = []
    ug1,vg1 = [], []
    ug2,vg2 = [], []

    geo1.append(diag_h_geop(thh[i], thb[i], z=-H / 2))
    geo2.append(diag_h_geop(thh[i], thb[i], z=H / 2))
    th1.append(diag_h_theta(thh[i], thb[i], z=-H / 2))
    th2.append(diag_h_theta(thh[i], thb[i], z=H / 2))
    vort1.append(diag_h_vorti(thh[i], thb[i], z=-H / 2))
    vort2.append(diag_h_vorti(thh[i], thb[i], z=H / 2))
    vv1.append(diag_h_vv(thh[i], thb[i], z=-H / 4))
    vv2.append(diag_h_vv(thh[i], thb[i], z=H / 4))   
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=-H / 2)
    ug1.append(ug_tmp)
    vg1.append(vg_tmp)
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=H / 2)
    ug2.append(ug_tmp)
    vg2.append(vg_tmp)
    mini.append(round(int(np.min(diag_h_geop(thh[i], thb[i], z=-H / 2)))))
    mini_ind.append(np.unravel_index(np.argmin(diag_h_geop(thh[i], thb[i], z=-H / 2),axis=None),thh[i].shape))
    
    geo12 = np.reshape(geo1, (MUTIL, NUTIL))
    geo22 = np.reshape(geo2, (MUTIL, NUTIL))
    th12 = np.reshape(th1, (MUTIL, NUTIL))
    th22 = np.reshape(th2, (MUTIL, NUTIL))
    vort12 = np.reshape(vort1, (MUTIL, NUTIL))
    vort22 = np.reshape(vort2, (MUTIL, NUTIL))
    vv12 = np.reshape(vv1, (MUTIL, NUTIL))
    vv22 = np.reshape(vv2, (MUTIL, NUTIL))
    ug12 = np.reshape(ug1, (MUTIL, NUTIL))
    vg12 = np.reshape(vg1, (MUTIL, NUTIL))
    ug22 = np.reshape(ug2, (MUTIL, NUTIL))
    vg22 = np.reshape(vg2, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(20,10))
    fig.suptitle('Coupes horizontales sol/tropopause : t='+str(steps[i]), fontsize=16)
    
    plt.subplot(221)
    plt.title("Géopotentiel température potentielle et vent au sol")
    plot_background()
    c = plt.contour(ord,abs,geo12,levels_geo_h,colors='k')
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    plt.quiver(ord[::5],abs[::5],vg12[::5, ::5],-ug12[::5, ::5],scale=500,headwidth=8,width=0.0013)

    plt.subplot(222)
    plt.title("Géopotentiel température potentielle et vent à la tropopause")
    plot_background()
    c = plt.contour(ord,abs,geo22,levels_geo_h,colors='k')
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    plt.quiver(ord[::5],abs[::5],vg22[::5, ::5],-ug22[::5, ::5],scale=500,headwidth=8,width=0.0013)
    
    plt.subplot(223)
    plt.title("Tourbillon géostrophique température potentielle et vitesse verticale au sol")
    plot_background()
    c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c1 = plt.contour(ord,abs,vort12,levels_vort_h_pos,colors="blue")
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv12,levels_vv_h_neg,colors="black",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv12,levels_vv_h_pos,colors="black",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    plt.subplot(224)
    plt.title("Tourbillon géostrophique température potentielle et vitesse verticale à la tropopause")
    plot_background()
    c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c1 = plt.contour(ord,abs,vort22,levels_vort_h_pos,colors="blue")
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv12,levels_vv_h_neg,colors="black",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv12,levels_vv_h_pos,colors="black",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    if steps[i]<10:
        figname=dir_figs+'Q4/Q4_h_00'+str(steps[i])
    if steps[i]>=10 and steps[i]<100:
        figname=dir_figs+'Q4/Q4_h_0'+str(steps[i])
    if steps[i]>=100:
        figname=dir_figs+'Q4/Q4_h_'+str(steps[i])
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.close()
    
print(mini)
print(mini_ind)

In [ ]:
dir_anim='./out/Q4/'
gif_filepath = dir_anim+'cyclo1.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Décrire la cyclogenèse observée.</p>
<p><b>2) </b>Afficher un graphique de l'évolution du minimum de géopotentiel (array mini) au cours de la simulation ainsi qu'un graphique du champ de géopotentiel au sol à la fin de la simulation (geo12) superposé au pointage de la trajectoire de la dépression (array mini_ind)</p> 
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Question 5 : Sensibilité du creusement au déphasage

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Afficher un graphique de la différence du minimum de géopotentiel entre la fin et le début de la  simulation pour différentes valeurs du déphasage initial (0 à 2400m par pas de 200m). En déduire la valeur du déphasage correspondant à un creusement optimal de la dépression.</p>
</div>

# Question 6 : Forçage géostrophique et vitesses verticales

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1400)
steps = [0, 48, 96, 144, 192, 240, 288, 336, 384]

thh, thb = Modele().run(thhi, thbi, steps)

for i in range(shape(thh)[0]):
    
    geo1 = []
    th1 = []
    vort1 = []
    vv1 = []
    modth1 = []
    ug1,vg1 = [], []
    q1,q2,divq1 = [], [], []
    
    geo1.append(diag_h_geop(thh[i], thb[i], z=-H / 2))
    th1.append(diag_h_theta(thh[i], thb[i], z=-H / 2))
    vort1.append(diag_h_vorti(thh[i], thb[i], z=-H / 2))
    vv1.append(diag_h_vv(thh[i], thb[i], z=-H / 4))
    modth1.append(diag_h_gradtheta(thh[i], thb[i], z=-H / 2))
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=-H / 2)
    ug1.append(ug_tmp)
    vg1.append(vg_tmp)
    divq1.append(diag_h_q(thh[i], thb[i], z=-H / 4)[2])
    q1.append(diag_h_q(thh[i], thb[i], z=-H / 4)[0])
    q2.append(diag_h_q(thh[i], thb[i], z=-H / 4)[1])
    
    geo12 = np.reshape(geo1, (MUTIL, NUTIL))
    th12 = np.reshape(th1, (MUTIL, NUTIL))
    vort12 = np.reshape(vort1, (MUTIL, NUTIL))
    vv12 = np.reshape(vv1, (MUTIL, NUTIL))
    modth12 = np.reshape(modth1, (MUTIL, NUTIL))
    ug12 = np.reshape(ug1, (MUTIL, NUTIL))
    vg12 = np.reshape(vg1, (MUTIL, NUTIL))
    divq12 = np.reshape(divq1, (MUTIL, NUTIL))
    q12 = np.reshape(q1, (MUTIL, NUTIL))
    q22 = np.reshape(q2, (MUTIL, NUTIL))

    fig = plt.figure(figsize=(12,20))
    fig.suptitle('Forçage géostrophique et vitesses verticales : t='+str(steps[i]), fontsize=16)
    
    plt.subplot(211)
    plt.title("Géopotentiel température potentielle et vent près du sol")
    plot_background()
    c1 = plt.contour(ord,abs,geo12,levels_geo_h,colors='k')
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c2 = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c2,c2.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    
    h1,_ = c1.legend_elements()
    h2,_ = c2.legend_elements()
    plt.legend([h1[0], h2[0]], ['Z', 'Theta'], loc='lower right')
    
    plt.quiver(ord[::5],abs[::5],vg12[::5, ::5],-ug12[::5, ::5],scale=500,headwidth=8,width=0.0013)
    
    plt.subplot(212)
    plt.title("Géopotentiel, température potentielle, vecteur Q et divergence, vitesse verticale près du sol")
    plot_background()
    c1 = plt.contour(ord,abs,geo12,levels_geo_h,colors='k')
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c2 = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c2,c2.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    #c2 = plt.contour(ord,abs,vort12,levels_vort_h_neg,colors="red")
    #plt.clabel(c2,c2.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv12,levels_vv_h_neg,colors="red",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv12,levels_vv_h_pos,colors="blue",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    h1,_ = c1.legend_elements()
    h2,_ = c2.legend_elements()
    h3,_ = c3.legend_elements()
    h4,_ = c4.legend_elements()
    plt.legend([h1[0], h2[0],h3[0], h4[0]], ['Z', 'Theta','VV-', 'VV+'], loc='lower right')
    
    cf = plt.contourf(ord,abs,divq12,levels_divQ_h,cmap="bwr")
    colord = plt.colorbar(cf, orientation="horizontal")
    colord.set_label("Divergence du vecteur Q")
    plt.quiver(ord[::2],abs[::2],q22[::2, ::2],-q12[::2, ::2],width=0.001
               ,headwidth=8,scale_units="width",scale=2e-10)
    
    if steps[i]<10:
        figname=dir_figs+'Q6/Q6_Q_00'+str(steps[i])
    if steps[i]>=10 and steps[i]<100:
        figname=dir_figs+'Q6/Q6_Q_0'+str(steps[i])
    if steps[i]>=100:
        figname=dir_figs+'Q6/Q6_Q_'+str(steps[i])
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
dir_anim='./out/Q6/'
gif_filepath = dir_anim+'forcage.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Utiliser ces graphiques pour expliquer le lien entre les vitesses verticales et le forçage géostrophique.</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Question 7 : Frontogenèse par déformation

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1400)
steps = [0, 48, 96, 144, 192, 240, 288, 336, 384]

thh, thb = Modele().run(thhi, thbi, steps)

mini = []

for i in range(shape(thh)[0]):
    
    geo1 = []
    geo2 = []
    th1 = []
    th2 = []
    vort1 = []
    vort2 = []
    vv1 = []
    vv2 = []
    modth1 = []
    modth2 = []
    ug1,vg1 = [], []
    ug2,vg2 = [], []
    def11,def21 = [], []
    def12,def22 = [], []

    geo1.append(diag_h_geop(thh[i], thb[i], z=-H / 2))
    th1.append(diag_h_theta(thh[i], thb[i], z=-H / 2))
    vort1.append(diag_h_vorti(thh[i], thb[i], z=-H / 2))
    vv1.append(diag_h_vv(thh[i], thb[i], z=-H / 4))
    modth1.append(diag_h_gradtheta(thh[i], thb[i], z=-H / 2))
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=-H / 2)
    ug1.append(ug_tmp)
    vg1.append(vg_tmp)
    def1_tmp, def2_tmp = diag_h_deform(thh[i], thb[i], z=-H / 2)
    def11.append(def1_tmp)
    def21.append(def2_tmp)
    
    geo12 = np.reshape(geo1, (MUTIL, NUTIL))
    th12 = np.reshape(th1, (MUTIL, NUTIL))
    vort12 = np.reshape(vort1, (MUTIL, NUTIL))
    vv12 = np.reshape(vv1, (MUTIL, NUTIL))
    modth12 = np.reshape(modth1, (MUTIL, NUTIL))
    ug12 = np.reshape(ug1, (MUTIL, NUTIL))
    vg12 = np.reshape(vg1, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(12,20))
    fig.suptitle('Frontogenèse par déformation : t='+str(steps[i]), fontsize=16)
    
    plt.subplot(211)
    plt.title("Tourbillon géostrophique et température potentielle près du sol")
    plot_background()
    c1 = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c2 = plt.contour(ord,abs,vort12,levels_vort_h_pos,colors="blue")
    plt.clabel(c2,c2.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    
    h1,_ = c1.legend_elements()
    h2,_ = c2.legend_elements()
    plt.legend([h1[0], h2[0]], ['Theta', 'Vort'], loc='lower right')
    
    plt.subplot(212)
    plt.title("Température potentielle (+module du gradient), déformation et vitesses verticales près du sol")
    plot_background()
    c1 = plt.contour(ord,abs,th12,levels_th_h,linewidth=1)
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    #c2 = plt.contour(ord,abs,vort12,levels_vort_h_neg,colors="red")
    #plt.clabel(c2,c2.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv12,levels_vv_h_neg,colors="grey",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv12,levels_vv_h_pos,colors="grey",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    h1,_ = c1.legend_elements()
    h3,_ = c3.legend_elements()
    h4,_ = c4.legend_elements()
    plt.legend([h1[0],h3[0], h4[0]], ['Theta','VV-', 'VV+'], loc='lower right')
    
    cf = plt.contourf(ord,abs,modth12,linspace(8e-8, 5e-5, 20),linewidth=1.5, cmap="hot_r")
    colord = plt.colorbar(cf, orientation="horizontal",format="%.1e")
    colord.set_label("Module du gradient de température potentielle $(K.m^{-1})$")
    plt.quiver(ord[::2],abs[::2],-def21[0][::2, ::2],def11[0][::2, ::2],scale=0.005,width=0.001,color="k")
    plt.quiver(ord[::2],abs[::2],def21[0][::2, ::2],-def11[0][::2, ::2],scale=0.005,width=0.001,color="k")
    
    if steps[i]<10:
        figname=dir_figs+'Q7/Q7_fronto_00'+str(steps[i])
    if steps[i]>=10 and steps[i]<100:
        figname=dir_figs+'Q7/Q7_fronto_0'+str(steps[i])
    if steps[i]>=100:
        figname=dir_figs+'Q7/Q7_fronto_'+str(steps[i])
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.close()

In [ ]:
dir_anim='./out/Q7/'
gif_filepath = dir_anim+'fronto.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Utiliser ces graphiques pour expliquer le mécansime de frontogénèse géostrophique.</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>

# Question 8 : Cyclogenèse avec une anomalie d'altitude uniquement

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1400)
thhi2, thbi2 = EtatInitial().gradeady_gradwernli()
steps = [0]
thh, thb = Modele().run(thhi, thbi2, steps)

for i in range(shape(thh)[0]):
    
    geo1 = []
    geo2 = []
    th1 = []
    th2 = []
    vort1 = []
    vort2 = []
    wind1 = []
    wind2 = []
    vv1 = []
    vv2 = []
    modth1 = []
    modth2 = []
    ug1,vg1 = [], []
    ug2,vg2 = [], []
    def11,def21 = [], []
    def12,def22 = [], []

    geo1.append(diag_h_geop(thh[i], thb[i], z=-H / 2))
    geo2.append(diag_h_geop(thh[i], thb[i], z=H / 2))
    th1.append(diag_h_theta(thh[i], thb[i], z=-H / 2))
    th2.append(diag_h_theta(thh[i], thb[i], z=H / 2))
    vort1.append(diag_h_vorti(thh[i], thb[i], z=-H / 2))
    vort2.append(diag_h_vorti(thh[i], thb[i], z=H / 2))
    wind1.append(diag_h_wind(thh[i], thb[i], z=-H / 2))
    wind2.append(diag_h_wind(thh[i], thb[i], z=H / 2))
    vv1.append(diag_h_vv(thh[i], thb[i], z=-H / 4))
    vv2.append(diag_h_vv(thh[i], thb[i], z=H / 4))
    modth1.append(diag_h_gradtheta(thh[i], thb[i], z=-H / 2))
    modth2.append(diag_h_gradtheta(thh[i], thb[i], z=H / 2))
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=-H / 2)
    ug1.append(ug_tmp)
    vg1.append(vg_tmp)
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=H / 2)
    ug2.append(ug_tmp)
    vg2.append(vg_tmp)
    def1_tmp, def2_tmp = diag_h_deform(thh[i], thb[i], z=-H / 2)
    def11.append(def1_tmp)
    def21.append(def2_tmp)
    def1_tmp, def2_tmp = diag_h_deform(thh[i], thb[i], z=-H / 2)
    def12.append(def1_tmp)
    def22.append(def2_tmp)
    
    geo12 = np.reshape(geo1, (MUTIL, NUTIL))
    geo22 = np.reshape(geo2, (MUTIL, NUTIL))
    th12 = np.reshape(th1, (MUTIL, NUTIL))
    th22 = np.reshape(th2, (MUTIL, NUTIL))
    vort12 = np.reshape(vort1, (MUTIL, NUTIL))
    vort22 = np.reshape(vort2, (MUTIL, NUTIL))
    vv12 = np.reshape(vv1, (MUTIL, NUTIL))
    vv22 = np.reshape(vv2, (MUTIL, NUTIL))
    modth12 = np.reshape(modth1, (MUTIL, NUTIL))
    modth22 = np.reshape(modth2, (MUTIL, NUTIL))
    ug12 = np.reshape(ug1, (MUTIL, NUTIL))
    vg12 = np.reshape(vg1, (MUTIL, NUTIL))
    ug22 = np.reshape(ug2, (MUTIL, NUTIL))
    vg22 = np.reshape(vg2, (MUTIL, NUTIL))
    def112 = np.reshape(def11, (MUTIL, NUTIL))
    def212 = np.reshape(def21, (MUTIL, NUTIL))
    def122 = np.reshape(def12, (MUTIL, NUTIL))
    def222 = np.reshape(def22, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(17,10))
    fig.suptitle('Etat initial du modèle : coupes horizontales sol/tropopause', fontsize=16)
    
    plt.subplot(221)
    plt.title("Géopotentiel température potentielle et vent au sol")
    plot_background()
    c = plt.contour(ord,abs,geo12,levels_geo_h,colors='k')
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    plt.quiver(ord[::5],abs[::5],vg12[::5, ::5],-ug12[::5, ::5],scale=500,headwidth=8,width=0.0013)
    
    plt.subplot(222)
    plt.title("Géopotentiel température potentielle et vent à la tropopause")
    plot_background()
    c = plt.contour(ord,abs,geo22,levels_geo_h,colors='k')
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    plt.quiver(ord[::5],abs[::5],vg22[::5, ::5],-ug22[::5, ::5],scale=500,headwidth=8,width=0.0013)
    
    plt.subplot(223)
    plt.title("Tourbillon géostrophique température potentielle et vitesse verticale au sol")
    plot_background()
    c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c1 = plt.contour(ord,abs,vort12,levels_vort_h_pos,colors="blue")
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv12,levels_vv_h_neg,colors="black",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv12,levels_vv_h_pos,colors="black",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    plt.subplot(224)
    plt.title("Tourbillon géostrophique et température potentielle et vitesse verticale à la tropopause")
    plot_background()
    c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c1 = plt.contour(ord,abs,vort22,levels_vort_h_pos,colors="blue")
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv22,levels_vv_h_neg,colors="black",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv22,levels_vv_h_pos,colors="black",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    figname=dir_figs+'Q8_init_h'
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.show()
    
    # Ajout d'une coupe zonale
    x = 4e6
    n_z = 11
    dz = H / (n_z - 1)
    z1 = range(0, int(H + dz), int(dz))
    
    theta = []
    vorti = []

    theta.append(transpose(diag_z_theta(thh[i], thb[i], x, n_z)))
    vorti.append(transpose(diag_z_vorti(thh[i], thb[i], x, n_z)))

    theta2 = np.reshape(theta, (n_z, NUTIL))
    vorti2 = np.reshape(vorti, (n_z, NUTIL))  

    n_z = 13
    dz = H / (n_z - 1)
    z2 = range(0, int(H + dz), int(dz))
    
    vvv = []
    vvv.append(transpose(diag_z_vv(thh[i], thb[i], x, n_z)))
    
    vvv2 = np.reshape(vvv, (n_z, NUTIL))
    
    fig = plt.figure(figsize=(10,8))
    fig.suptitle('Etat initial du modèle : coupe zonale', fontsize=16)
    plt.subplot(111)
    plt.title("Température potentielle tourbillon géostrophique et vitesse verticale")
    plt.ylim([0, 10])
    plt.xlabel("Axe Ouest - Est ($km$)", fontsize=10)
    plt.xticks(np.arange(NUTIL)[::12],ord[::12])
    plt.ylabel("Altitude ($km$)", fontsize=10)
    c = plt.contour(theta2,levels=levels_th_m,linewidths=2)
    plt.clabel(c, fontsize=8, fmt="%1.0f")
    #c1 = plt.contour(vorti2,levels=levels_vort_z_neg,colors='red',linewidths=2)
    #plt.clabel(c1, c1.levels[::2], fontsize=8, fmt="%.0e")
    c2 = plt.contour(vorti2,levels=levels_vort_z_pos,colors='blue',linewidths=2)
    plt.clabel(c2, c2.levels[::2], fontsize=8, fmt="%.0e")
    
    #plt.quiver(abs[::4],z2[::4],vvv2[::4, ::4] *1e-50,vvv2[::4, ::4] *1000,scale=50,width=0.005,headwidth=3,scale_units="width")
    c = plt.contour(vvv2, colors='k')
    plt.clabel(c, c.levels[::2], fontsize=7, fmt="%.0e")
    
    figname=dir_figs+'Q8_init_z'
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.show()

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1400)
thhi2, thbi2 = EtatInitial().gradeady_gradwernli()
steps = [0, 48, 96, 144, 192, 240, 288, 336, 384]
thh, thb = Modele().run(thhi, thbi2, steps)

mini2 = []
mini_ind2 = []

for i in range(shape(thh)[0]):
    
    geo1 = []
    geo2 = []
    th1 = []
    th2 = []
    vort1 = []
    vort2 = []
    vv1 = []
    vv2 = []
    ug1,vg1 = [], []
    ug2,vg2 = [], []

    geo1.append(diag_h_geop(thh[i], thb[i], z=-H / 2))
    geo2.append(diag_h_geop(thh[i], thb[i], z=H / 2))
    th1.append(diag_h_theta(thh[i], thb[i], z=-H / 2))
    th2.append(diag_h_theta(thh[i], thb[i], z=H / 2))
    vort1.append(diag_h_vorti(thh[i], thb[i], z=-H / 2))
    vort2.append(diag_h_vorti(thh[i], thb[i], z=H / 2))
    vv1.append(diag_h_vv(thh[i], thb[i], z=-H / 4))
    vv2.append(diag_h_vv(thh[i], thb[i], z=H / 4))   
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=-H / 2)
    ug1.append(ug_tmp)
    vg1.append(vg_tmp)
    ug_tmp, vg_tmp = diag_h_wind(thh[i], thb[i], z=H / 2)
    ug2.append(ug_tmp)
    vg2.append(vg_tmp)
    mini2.append(round(int(np.min(diag_h_geop(thh[i], thb[i], z=-H / 2)))))
    mini_ind2.append(np.unravel_index(np.argmin(diag_h_geop(thh[i], thb[i], z=-H / 2),axis=None),thh[i].shape))
    
    geo12 = np.reshape(geo1, (MUTIL, NUTIL))
    geo22 = np.reshape(geo2, (MUTIL, NUTIL))
    th12 = np.reshape(th1, (MUTIL, NUTIL))
    th22 = np.reshape(th2, (MUTIL, NUTIL))
    vort12 = np.reshape(vort1, (MUTIL, NUTIL))
    vort22 = np.reshape(vort2, (MUTIL, NUTIL))
    vv12 = np.reshape(vv1, (MUTIL, NUTIL))
    vv22 = np.reshape(vv2, (MUTIL, NUTIL))
    ug12 = np.reshape(ug1, (MUTIL, NUTIL))
    vg12 = np.reshape(vg1, (MUTIL, NUTIL))
    ug22 = np.reshape(ug2, (MUTIL, NUTIL))
    vg22 = np.reshape(vg2, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(20,10))
    fig.suptitle('Coupes horizontales sol/tropopause : t='+str(steps[i]), fontsize=16)
    
    plt.subplot(221)
    plt.title("Géopotentiel température potentielle et vent au sol")
    plot_background()
    c = plt.contour(ord,abs,geo12,levels_geo_h,colors='k')
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    plt.quiver(ord[::5],abs[::5],vg12[::5, ::5],-ug12[::5, ::5],scale=500,headwidth=8,width=0.0013)

    plt.subplot(222)
    plt.title("Géopotentiel température potentielle et vent à la tropopause")
    plot_background()
    c = plt.contour(ord,abs,geo22,levels_geo_h,colors='k')
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
    c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    plt.quiver(ord[::5],abs[::5],vg22[::5, ::5],-ug22[::5, ::5],scale=500,headwidth=8,width=0.0013)
    
    plt.subplot(223)
    plt.title("Tourbillon géostrophique température potentielle et vitesse verticale au sol")
    plot_background()
    c = plt.contour(ord,abs,th12,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c1 = plt.contour(ord,abs,vort12,levels_vort_h_pos,colors="blue")
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv12,levels_vv_h_neg,colors="black",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv12,levels_vv_h_pos,colors="black",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    plt.subplot(224)
    plt.title("Tourbillon géostrophique température potentielle et vitesse verticale à la tropopause")
    plot_background()
    c = plt.contour(ord,abs,th22,levels_th_h,linewidth=1.5)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=8,fmt="%1.0f")
    c1 = plt.contour(ord,abs,vort22,levels_vort_h_pos,colors="blue")
    plt.clabel(c1,c1.levels[::2],inline=1,fontsize=6,fmt="%1.6f")
    c3 = plt.contour(ord,abs,vv12,levels_vv_h_neg,colors="black",linestyles="--")
    plt.clabel(c3,c3.levels[::2],inline=1,fontsize=6,fmt="%1.3f")
    c4 = plt.contour(ord,abs,vv12,levels_vv_h_pos,colors="black",linestyles="-")
    plt.clabel(c4,c4.levels[::2],fontsize=6,fmt="%1.3f")
    
    if steps[i]<10:
        figname=dir_figs+'Q8/Q8_h_00'+str(steps[i])
    if steps[i]>=10 and steps[i]<100:
        figname=dir_figs+'Q8/Q8_h_0'+str(steps[i])
    if steps[i]>=100:
        figname=dir_figs+'Q8/Q8_h_'+str(steps[i])
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.close()
    
print(mini2)

In [ ]:
dir_anim='./out/Q8/'
gif_filepath = dir_anim+'cyclo2.gif'
make_animation()
IPdisplay.Image(url=gif_filepath)

<div class="alert alert-warning">
<b>Questions : </b>
<p><b>1) </b>Une cyclogenèse est-elle possible en présence d'une anomalie d'altitude uniquement ? Par quel mécanisme ?
<p><b>2) </b>Afficher un graphique de l'évolution du minimum de géopotentiel (array mini2) au cours de la simulation ainsi qu'un graphique du champ de géopotentiel au sol à la fin de la simulation (geo12) superposé au pointage de la trajectoire de la dépression (array mini_ind2)
</p>
<p><b>3) </b>Comparer le cresuement avec le cas avec deux anomalies.
</p>
</div>

<div class="alert alert-success">
<b>Réponses : </b>
</div>

# Question 9 : Comparaisons

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1700)
thh0, thb0 = Modele().run(thhi, thbi, steps=[0])

thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=1700)
thh1, thb1 = Modele().run(thhi, thbi, steps=[288])

for i in range(shape(thh0)[0]):
    
    geo = []
    geo.append(diag_h_geop(thh0[i], thb0[i], z=-H / 2))
    geo2 = np.reshape(geo, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(15,8))
    fig.suptitle('Simulation avec déphasage de 1700m', fontsize=16)
    
    plt.subplot(111)
    plt.title("Géopotentiel au sol")
    plot_background()
    c = plt.contour(ord,abs,geo2,levels_geo_h)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
  
    figname=dir_figs+'Q9_compar1a'
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.show()
    
for i in range(shape(thh1)[0]):
    
    geo = []
    geo.append(diag_h_geop(thh1[i], thb1[i], z=-H / 2))
    geo2 = np.reshape(geo, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(15,8))
    fig.suptitle('Simulation avec déphasage de 1700m', fontsize=16)
    
    plt.subplot(111)
    plt.title("Géopotentiel au sol")
    plot_background()
    c = plt.contour(ord,abs,geo2,levels_geo_h)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
  
    figname=dir_figs+'Q9_compar1b'
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.show()

In [ ]:
thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=5500)
thh0, thb0 = Modele().run(thhi, thbi, steps=[0])

thhi, thbi = EtatInitial().gradeady_gradwernli_anotripo(tilt=5500)
thh1, thb1 = Modele().run(thhi, thbi, steps=[288])

for i in range(shape(thh0)[0]):
    
    geo = []
    geo.append(diag_h_geop(thh0[i], thb0[i], z=-H / 2))
    geo2 = np.reshape(geo, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(15,8))
    fig.suptitle('Simulation avec déphasage de 5500m', fontsize=16)
    
    plt.subplot(111)
    plt.title("Géopotentiel au sol")
    plot_background()
    c = plt.contour(ord,abs,geo2,levels_geo_h)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
  
    figname=dir_figs+'Q9_compar2a'
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.show()
    
for i in range(shape(thh1)[0]):
    
    geo = []
    geo.append(diag_h_geop(thh1[i], thb1[i], z=-H / 2))
    geo2 = np.reshape(geo, (MUTIL, NUTIL))
    
    fig = plt.figure(figsize=(15,8))
    fig.suptitle('Simulation avec déphasage de 5500m', fontsize=16)
    
    plt.subplot(111)
    plt.title("Géopotentiel au sol")
    plot_background()
    c = plt.contour(ord,abs,geo2,levels_geo_h)
    plt.clabel(c,c.levels[::2],inline=1,fontsize=10,fmt="%1.0f")
  
    figname=dir_figs+'Q9_compar2b'
    plt.savefig(figname+'.png',bbox_inches='tight')
    plt.show()

<div class="alert alert-warning">
<b>Question : </b>
<p><b>1) </b>Comment expliquer les différences observées entre ces deux simulations ?</p>
</div>

<div class="alert alert-success">
<b>Réponse : </b>
</div>